In [2]:
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string
from collections import Counter
from heapq import nlargest
import os


2023-06-21 16:31:27.775660: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-06-21 16:31:29.472224: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-06-21 16:31:29.472294: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-06-21 16:31:29.752749: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-06-21 16:31:32.688718: W tensorflow/stream_executor/platform/de

In [3]:
#pull the data
url = os.getcwd() + '/raw_data/hotel_reviews.csv'
raw_df = pd.read_csv(url)

In [10]:
raw_df.columns

Index(['Hotel_Address', 'Additional_Number_of_Scoring', 'Review_Date',
       'Average_Score', 'Hotel_Name', 'Reviewer_Nationality',
       'Negative_Review', 'Review_Total_Negative_Word_Counts',
       'Total_Number_of_Reviews', 'Positive_Review',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score', 'Tags',
       'days_since_review', 'lat', 'lng'],
      dtype='object')

In [4]:
review_df = pd.DataFrame(pd.concat((raw_df['Negative_Review'], raw_df['Positive_Review']), axis=0), columns=['reviews'])
len(review_df)

1031476

In [16]:
reviews_str = review_df['reviews'].head(20).str.cat(sep=' ')

In [20]:
spacy_model = spacy.load('en_core_web_sm')
doc = spacy_model(reviews_str)

In [21]:
len(list(doc)), len(reviews_str)

(1211, 6037)

In [22]:
#cleaning
keyword = []
stopwords = list(STOP_WORDS)
post_tag = ['PROPN', 'ADJ', 'VERB', 'NOUN']
for token in doc:
    if token.text in stopwords or token.text in string.punctuation:
        continue
    if token.pos_ in post_tag:
        keyword.append(token.text)

In [25]:
freq_keyword = Counter(keyword)
freq_keyword.most_common(5)

[('room', 26), ('t', 10), ('hotel', 9), ('floor', 7), ('rooms', 7)]

In [26]:
max_freq = freq_keyword.most_common(5)[0][1]
for word in freq_keyword.keys():
    freq_keyword[word] = freq_keyword[word]/max_freq
freq_keyword.most_common(5)

[('room', 1.0),
 ('t', 0.38461538461538464),
 ('hotel', 0.34615384615384615),
 ('floor', 0.2692307692307692),
 ('rooms', 0.2692307692307692)]

In [36]:
len(doc), len(list(doc.sents)[0])

(1211, 182)

In [48]:
sent_strength = {}
for sent in doc.sents:
    for word in sent:
        if word.text in freq_keyword.keys():
            if sent in sent_strength.keys():
                sent_strength[sent] += freq_keyword[word.text]
            else:
                sent_strength[sent] = freq_keyword[word.text]

<class 'dict'>


In [52]:
sent_strength.get

<function dict.get(key, default=None, /)>

In [62]:
summarized_sentences = nlargest(3, sent_strength, key=sent_strength.get)

In [63]:
final_sentences = [w.text for w in summarized_sentences]
summary = ' '.join(final_sentences)
print(len(summary))
print(summary)

3200
My room was dirty and I was afraid to walk barefoot on the floor which looked as if it was not cleaned in weeks White furniture which looked nice in pictures was dirty too and the door looked like it was attacked by an angry dog My shower drain was clogged and the staff did not respond to my request to clean it On a day with heavy rainfall a pretty common occurrence in Amsterdam the roof in my room was leaking luckily not on the bed you could also see signs of earlier water damage I also saw insects running on the floor Overall the second floor of the property looked dirty and badly kept On top of all of this a repairman who came to fix something in a room next door at midnight was very noisy as were many of the guests I understand the challenges of running a hotel in an old building but this negligence is inconsistent with prices demanded by the hotel On the last night after I complained about water damage the night shift manager offered to move me to a different room but that of